In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import optuna
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

category_columns = [
    "Unlawful detention",
    "Human trafficking",
    "Enslavement",
    "Willful killing of civilians",
    "Mass execution",
    "Kidnapping",
    "Extrajudicial killing",
    "Forced disappearance",
    "Damage or destruction of civilian critical infrastructure",
    "Damage or destruction, looting, or theft of cultural heritage",
    "Military operations (battle, shelling)",
    "Gender-based or other conflict-related sexual violence",
    "Violent crackdowns on protesters/opponents/civil rights abuse",
    "Indiscriminate use of weapons",
    "Torture or indications of torture",
    "Persecution based on political, racial, ethnic, gender, or sexual orientation",
    "Movement of military, paramilitary, or other troops and equipment"
]

# 2) Custom Dataset class for articles
class ArticleDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

# Load datasets
train_df = pd.read_csv("train.csv")  
val_df   = pd.read_csv("val.csv")    
test_df  = pd.read_csv("test.csv")  

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the text columns
train_encodings = tokenizer(
    list(train_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)
val_encodings   = tokenizer(
    list(val_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)
test_encodings  = tokenizer(
    list(test_df["Incident Narrative"].values),
    truncation=True,
    padding=True
)

# Extract labels (multi-label targets in your category columns)
train_labels = train_df[category_columns].values
val_labels   = val_df[category_columns].values
test_labels  = test_df[category_columns].values

# Create Dataset objects
train_dataset = ArticleDataset(train_encodings, train_labels)
val_dataset   = ArticleDataset(val_encodings, val_labels)
test_dataset  = ArticleDataset(test_encodings, test_labels)

# Define compute_metrics for multi-label classification
def compute_metrics(p):
    preds = torch.sigmoid(torch.tensor(p.predictions))  # Convert logits to probabilities
    preds = (preds > 0.5).int().cpu().numpy() 
    labels = torch.tensor(p.label_ids).cpu().numpy()
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels,
        preds,
        average='weighted'
    )
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Optuna hyperparameter optimization
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    weight_decay = trial.suggest_uniform('weight_decay', 0.01, 0.1)
    num_train_epochs = trial.suggest_int('num_train_epochs', 5, 20)

    # Define model and training arguments
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased", 
        num_labels=len(category_columns)
    )

    training_args = TrainingArguments(
        output_dir='./results', 
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        logging_dir='./logs',
        logging_steps=10,
        disable_tqdm=True
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    # Train and evaluate the model
    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_f1"]

# Run Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)

# Train the final model with the best hyperparameters
final_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels=len(category_columns)
)
final_training_args = TrainingArguments(
    output_dir='./final_results', 
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_params['learning_rate'],
    per_device_train_batch_size=best_params['batch_size'],
    per_device_eval_batch_size=best_params['batch_size'],
    num_train_epochs=best_params['num_train_epochs'],
    weight_decay=best_params['weight_decay'],
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir='./final_logs'
)

trainer = Trainer(
    model=final_model,
    args=final_training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

# Evaluate the final model on the test dataset
test_results = trainer.evaluate(test_dataset)
print("Final Test Results:", test_results)


c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-01-03 10:55:30,218] A new study created in memory with name: no-name-d6c61be0-db6f-4ff7-bdcf-7540abe5c331
C:\Users\ymasr\AppData\Local\Temp\ipykernel_14764\3822549815.py:99: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 5e-5)
C:\Users\ymasr\AppData\Local\Temp\ipykernel_14764\3822549815.py:101: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float 

{'loss': 0.6259, 'grad_norm': 1.746819019317627, 'learning_rate': 1.89132482280482e-05, 'epoch': 0.24390243902439024}
{'loss': 0.5156, 'grad_norm': 1.785358190536499, 'learning_rate': 1.860063255485732e-05, 'epoch': 0.4878048780487805}
{'loss': 0.4391, 'grad_norm': 0.9706712961196899, 'learning_rate': 1.828801688166644e-05, 'epoch': 0.7317073170731707}
{'loss': 0.3827, 'grad_norm': 0.8255770802497864, 'learning_rate': 1.797540120847556e-05, 'epoch': 0.975609756097561}


c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to

{'eval_loss': 0.35463032126426697, 'eval_accuracy': 0.017543859649122806, 'eval_f1': 0.046153846153846156, 'eval_precision': 0.2, 'eval_recall': 0.02608695652173913, 'eval_runtime': 0.7783, 'eval_samples_per_second': 73.232, 'eval_steps_per_second': 10.278, 'epoch': 1.0}
{'loss': 0.3484, 'grad_norm': 0.6654301285743713, 'learning_rate': 1.7662785535284682e-05, 'epoch': 1.2195121951219512}
{'loss': 0.3284, 'grad_norm': 0.7192702889442444, 'learning_rate': 1.7350169862093803e-05, 'epoch': 1.4634146341463414}
{'loss': 0.2982, 'grad_norm': 0.7436495423316956, 'learning_rate': 1.7037554188902925e-05, 'epoch': 1.7073170731707317}
{'loss': 0.2945, 'grad_norm': 0.5854768753051758, 'learning_rate': 1.6724938515712043e-05, 'epoch': 1.951219512195122}


c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to

...

{'eval_loss': 0.1758529245853424, 'eval_accuracy': 0.3508771929824561, 'eval_f1': 0.6538582713851937, 'eval_precision': 0.7104661039443647, 'eval_recall': 0.6434782608695652, 'eval_runtime': 0.7001, 'eval_samples_per_second': 81.416, 'eval_steps_per_second': 11.427, 'epoch': 17.0}


100%|██████████| 697/697 [05:53<00:00,  1.97it/s]


{'train_runtime': 353.8693, 'train_samples_per_second': 15.517, 'train_steps_per_second': 1.97, 'train_loss': 0.16374067254524832, 'epoch': 17.0}


 67%|██████▋   | 4/6 [00:00<00:00, 28.13it/s]c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ymasr\OneDrive\Desktop\projects\BERT_GMU_Classiifcation\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predic

Final Test Results: {'eval_loss': 0.1783057302236557, 'eval_accuracy': 0.3488372093023256, 'eval_f1': 0.6300961248928729, 'eval_precision': 0.726111111111111, 'eval_recall': 0.5777777777777777, 'eval_runtime': 0.2622, 'eval_samples_per_second': 163.981, 'eval_steps_per_second': 22.881, 'epoch': 17.0}
